In [16]:
import glob
import pandas as pd
import re

In [154]:
directorio_json = "/Users/rudikrompich/Git/analisis_tse_elecciones_2023/departamento/distrito_central/municipio/guatemala/corporacion_municipal/actas_originales"
directorio_json_documentos = glob.glob(directorio_json + "/*.json")
directorio_trep_documento = "/Users/rudikrompich/Git/analisis_tse_elecciones_2023/departamento/distrito_central/municipio/guatemala/corporacion_municipal/actas_csv_sistema_trep/4-CORPORACIÓN/gtm2023_e4d00m01.csv"


In [155]:
df_complete_ocr = pd.DataFrame()
for documento in directorio_json_documentos:
    parts = documento.split('/')
    desired_part = parts[-1]
    numbers = re.findall(r'\d+', desired_part)
    centro = int(numbers[0])
    mesa = int(numbers[1])
    df = pd.read_json(documento)
    df["CENTRO_DE_VOTACIÓN"] = centro
    df["MESA"] = mesa
    df_complete_ocr = pd.concat([df_complete_ocr, df])
df_complete_ocr.columns =  ['PARTIDO', 'CANTIDAD_OCR', 'CENTRO_DE_VOTACIÓN', 'MESA']

df_complete_ocr = df_complete_ocr[(df_complete_ocr["PARTIDO"]!="Total Votos Válidos") & (df_complete_ocr["PARTIDO"]!="Votos Nulos")& (df_complete_ocr["PARTIDO"]!="Votos en Blanco")&(df_complete_ocr["PARTIDO"]!="Total Votos Válidamente Emitidos")&(df_complete_ocr["PARTIDO"]!="Votos Inválidos")& (df_complete_ocr["PARTIDO"]!="Total Votos")&(df_complete_ocr["PARTIDO"]!="Impugnaciones")&(df_complete_ocr["PARTIDO"]!="Cantidad de papeletas recibidas")&(df_complete_ocr["PARTIDO"]!="Cantidad de papeletas no usadas")].copy().reset_index(drop=True)

diccionario_partidos = {'UNIDAD NACIONAL DE LA ESPERANZA':'UNE',
 'VALOR UNIONISTA':'VALOR UNIONISTA',
 'PARTIDO REPUBLICANO':'PR',
 'CAMBIO':'CAMBIO',
 'PARTIDO AZUL':'AZUL',
 'MOVIMIENTO PARA LA LIBERACION DE LOS PUEBLOS':'MLP',
 'VICTORIA':'VICTORIA',
 'VAMOS POR UNA GUATEMALA DIFERENTE':'VAMOS',
 'PARTIDO HUMANISTA DE GUATEMALA':'PHG',
 'BIENESTAR NACIONAL':'BIEN',
 'UNION REPUBLICANA':'UR',
 'PARTIDO DE AVANZADA NACIONAL':'P A N',
 'PARTIDO DE INTEGRACIÓN NACIONAL':'PIN',
 'FRENTE DE CONVERGENCIA NACIONAL':'FCN-NACION',
 'COMITE CIVICO LA GOTA DE AGUA':'COMITE CIVICO LA GOTA DE AGUA',
 'PARTIDO POLITICO NOSOTROS':'PPN',
 'SEMILLA URNG-MAIZ WINAQ':'FOPPA X LA CIUDAD',
 'MI FAMILIA':'MI FAMILIA',
 'CABAL':'CABAL',
 'VISION CON VALORES':'VIVA',
 'PODEMOS':'PODEMOS',
 'PARTIDO DE OPORTUNIDADES Y DESARROLLO':'PODER',
 'COMPROMISO, RENOVACION Y ORDEN': 'CREO'}

df_complete_ocr["PARTIDO"] = df_complete_ocr["PARTIDO"].replace(diccionario_partidos)

df_complete_ocr

,PARTIDO,CANTIDAD_OCR,CENTRO_DE_VOTACIÓN,MESA
0,UNE,17,94,1276
1,VALOR UNIONISTA,36,94,1276
2,PR,0,94,1276
3,CAMBIO,4,94,1276
4,AZUL,3,94,1276
...,...,...,...,...
46524,CABAL,4,42,537
46525,VIVA,7,42,537
46526,PODEMOS,14,42,537
46527,PODER,0,42,537


In [156]:
#df_complete_ocr = df_complete_ocr[(df_complete_ocr["PARTIDO"]=="VALOR UNIONISTA") | (df_complete_ocr["PARTIDO"]=="COMPROMISO, RENOVACION Y ORDEN")| (df_complete_ocr["PARTIDO"]=="SEMILLA URNG-MAIZ WINAQ")].copy().reset_index(drop=True)

In [157]:
df_complete_trep = pd.read_csv(directorio_trep_documento, sep=",",skiprows=4)
df_complete_trep = df_complete_trep.iloc[:,:31].copy().drop(columns=["DEPARTAMENTO", "ID_DEPARTAMENTO", "MUNICIPIO", "ID_MUNICIPIO", "PADRÓN"])
df_complete_trep = pd.melt(df_complete_trep, id_vars=["CENTRO_DE_VOTACIÓN", "MESA"], value_vars=df_complete_trep.columns[2:], var_name="PARTIDO", value_name="CANTIDAD_TREP")

df_complete_trep["CANTIDAD_TREP"] = df_complete_trep["CANTIDAD_TREP"].replace("--", 0)
df_complete_trep["CANTIDAD_TREP"] = df_complete_trep["CANTIDAD_TREP"].astype(int)
df_complete_trep

,CENTRO_DE_VOTACIÓN,MESA,PARTIDO,CANTIDAD_TREP
0,1,1,TODOS,9
1,1,2,TODOS,14
2,1,3,TODOS,10
3,1,4,TODOS,9
4,2,5,TODOS,7
...,...,...,...,...
48547,152,2019,CREO,2
48548,152,2020,CREO,1
48549,152,2021,CREO,2
48550,152,2022,CREO,4


In [158]:
df_complete_ocr.groupby(["PARTIDO"]).sum()

,CANTIDAD_OCR,CENTRO_DE_VOTACIÓN,MESA
PARTIDO,,,
AZUL,11760,152812,2047276
BIEN,5557,152812,2047276
CABAL,4840,152812,2047276
CAMBIO,6111,152812,2047276
COMITE CIVICO LA GOTA DE AGUA,1127,152812,2047276
CREO,105698,152812,2047276
FCN-NACION,1555,152812,2047276
FOPPA X LA CIUDAD,77663,152812,2047276
MI FAMILIA,2278,152812,2047276


In [159]:
df_complete_trep.groupby(["PARTIDO"]).sum()


,CENTRO_DE_VOTACIÓN,MESA,CANTIDAD_TREP
PARTIDO,,,
AZUL,152812,2047276,11760
BIEN,152812,2047276,5557
CABAL,152812,2047276,4840
CAMBIO,152812,2047276,6111
COMITE CIVICO LA GOTA DE AGUA,152812,2047276,1127
CREO,152812,2047276,105698
FCN-NACION,152812,2047276,1555
FOPPA X LA CIUDAD,152812,2047276,77663
MI FAMILIA,152812,2047276,2278


In [149]:
df_completo_con_ocr_y_trep = pd.merge(df_complete_ocr, df_complete_trep, on=["CENTRO_DE_VOTACIÓN", "MESA", "PARTIDO"],how='left')
df_completo_con_ocr_y_trep

,PARTIDO,CANTIDAD_OCR,CENTRO_DE_VOTACIÓN,MESA,CANTIDAD_TREP
0,UNE,17,94,1276,17
1,VALOR UNIONISTA,36,94,1276,36
2,PR,0,94,1276,0
3,CAMBIO,4,94,1276,4
4,AZUL,3,94,1276,3
...,...,...,...,...,...
46524,CABAL,4,42,537,4
46525,VIVA,7,42,537,7
46526,PODEMOS,14,42,537,14
46527,PODER,0,42,537,0


In [150]:
df_completo_con_ocr_y_trep

,PARTIDO,CANTIDAD_OCR,CENTRO_DE_VOTACIÓN,MESA,CANTIDAD_TREP
0,UNE,17,94,1276,17
1,VALOR UNIONISTA,36,94,1276,36
2,PR,0,94,1276,0
3,CAMBIO,4,94,1276,4
4,AZUL,3,94,1276,3
...,...,...,...,...,...
46524,CABAL,4,42,537,4
46525,VIVA,7,42,537,7
46526,PODEMOS,14,42,537,14
46527,PODER,0,42,537,0


In [151]:
df_completo_con_ocr_y_trep["DIFERENCIA"] = df_completo_con_ocr_y_trep["CANTIDAD_OCR"] - df_completo_con_ocr_y_trep["CANTIDAD_TREP"]

In [152]:
df_final_diferencias = df_completo_con_ocr_y_trep[df_completo_con_ocr_y_trep["DIFERENCIA"] != 0].copy()


In [153]:
df_final_diferencias

,PARTIDO,CANTIDAD_OCR,CENTRO_DE_VOTACIÓN,MESA,CANTIDAD_TREP,DIFERENCIA


In [116]:
#df_final_diferencias.to_csv("/Users/rudikrompich/Git/analisis_tse_elecciones_2023/departamento/distrito_central/municipio/guatemala/corporacion_municipal/diferencias_ocr_trep/diferencias.csv", index=False)